In [1]:
from google.colab import drive
import pandas

In [2]:
drive.mount('/content/drive', force_remount=True)
data_dir = "/content/drive/My Drive/Inzynierka/Dane/"

Mounted at /content/drive


In [3]:
competition = "accelerometer-biometric-competition.zip"

In [4]:
# # importing the zipfile module
# from zipfile import ZipFile

# # loading the temp.zip and creating a zip object
# with ZipFile(data_dir+"Competition/train.csv.zip", 'r') as zObject:

#     # Extracting all the members of the zip
#     # into a specific location.
#     zObject.extractall(
#         path=data_dir+"Competition")

In [5]:
csv_train = data_dir+"Competition/train.csv"
csv_test = data_dir+"Competition/test.csv"

In [6]:
import pandas as pd

columns = ['X','Y','Z','Device']
df_train = pd.read_csv(csv_train, usecols=columns)
# columns = ['X','Y','Z','SequenceId']
# df_test = pd.read_csv(csv_test, usecols=columns)
# df_test.rename(columns={'SequenceId':'Device'}, inplace=True)

In [7]:
print(df_train.head())
# print(df_test.head())

          X         Y         Z  Device
0  0.340509  8.308413  4.140585       7
1  0.381370  8.390134  4.249548       7
2  0.272407  8.471856  4.018002       7
3  0.149824  8.430995  4.290409       7
4  0.272407  8.430995  4.481094       7


In [8]:
print(df_train.Device.unique())
# print(df_test.Device.unique())

[   7    8    9   12   23   25   26   27   33   37   39   45   47   51
   52   57   58   65   67   68   70   71   73   74   75   78   79   81
   87   89   90   91   92   94   95   96   99  104  105  108  110  111
  116  117  120  122  124  126  127  129  134  137  142  145  148  149
  152  156  157  158  159  162  163  168  169  174  175  177  183  187
  188  189  190  194  196  204  206  207  211  213  216  219  222  224
  229  232  233  234  236  237  239  240  261  263  268  269  270  271
  273  274  275  277  281  282  283  284  285  289  290  291  294  296
  297  298  299  302  306  309  312  313  314  323  325  333  335  338
  341  343  344  345  350  360  361  366  369  370  371  376  378  381
  390  394  398  399  401  404  411  412  413  415  417  421  422  423
  425  433  438  447  448  455  461  463  466  471  473  477  478  479
  482  485  486  487  491  492  494  501  503  505  507  509  514  515
  518  520  523  524  528  531  533  534  536  537  539  547  550  552
  553 

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_features = scaler.fit_transform(df_train.drop('Device', axis=1))
# df_test_features = scaler.transform(df_test.drop('Device', axis=1))

In [10]:
print(df_train_features)
# print(df_test_features)

[[-0.09228431  1.03579976 -0.12475962]
 [-0.08250146  1.05270035 -0.10268157]
 [-0.10858907  1.06960115 -0.14959726]
 ...
 [-0.210439   -0.74573331 -2.95081588]
 [-0.2470699  -0.777375   -2.85761079]
 [-0.2837008  -0.84086518 -2.95081588]]


In [11]:
df_train[['X', 'Y', 'Z']] = df_train_features

In [13]:
seq_length = 300

def balance_device_rows(df, target_count):
    result = []
    for device, group in df.groupby('Device'):
        repeated_group = group.sample(target_count, random_state=1)
        result.append(repeated_group)
    return pd.concat(result).reset_index(drop=True)

In [14]:
device_counts = df_train['Device'].value_counts(sort=False)
print(device_counts)

Device
7       523187
8       324429
9       149495
12       10468
23      353344
         ...  
1031      3271
1033    136455
1035     22111
1036     50638
1037    263110
Name: count, Length: 387, dtype: int64


In [16]:
# Max sekwencji

# sequences_count = []
# # data = []
# device_unique = df_train.Device.unique()
# data = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
# for device in device_unique:
#   index = df_train.loc[df_train['Device'] == device].index[0]
#   n = device_counts[device] // 300
#   sequences_count.append(n)
#   data = pd.concat([data, df_train.iloc[index:index+n*300]], ignore_index=True)

In [17]:
# Po 10 sekwencji

sequences_count = []
# data = []
n = 10
device_unique = df_train.Device.unique()
data = pd.DataFrame({'X': [], 'Y': [],'Z': [], 'Device': []})
for device in device_unique:
  index = df_train.loc[df_train['Device'] == device].index[0]
  # n = device_counts[device] // 300
  sequences_count.append(n)
  data = pd.concat([data, df_train.iloc[index:index+n*300]], ignore_index=True)

In [18]:
sequences = 0
for i in sequences_count:
  sequences += i
print(sequences)

3870


#Sieć neuronowa

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [26]:
X = data[['X', 'Y', 'Z']].to_numpy()

In [27]:
def create_sequences(data, seq_length):
    X = []
    for i in range (int(len(data) / seq_length)):
        X.append(data[i:i+seq_length])
    return np.array(X)

In [28]:
X_seq= create_sequences(X, seq_length)

In [31]:
y_seq = []
c = 0
for i in sequences_count:
  a = device_unique[c]
  for j in range(i):
    y_seq.append(a)
  # print(a)
  c +=1
y_seq = np.array(y_seq)

In [32]:
y_seq.shape

(3870,)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_seq,y_seq ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [34]:
for i in range (0, len(y_train)) :
  y_train[i] = str(y_train[i])

In [35]:
y_train

array([207, 793, 801, ..., 190, 491,  26])

In [37]:
from sklearn.preprocessing import OneHotEncoder

data_reshaped = np.array(y_train).reshape(-1, 1)
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data_reshaped)
encoded_test = encoder.transform(np.array(y_test).reshape(-1, 1))

In [38]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# # Shuffle and batch the data
# batch_size = 32  # You can set your preferred batch size
# train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

In [44]:
model = Sequential()
model.add(LSTM(units=128, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=387, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X_train, encoded_data, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 37s 382ms/step - accuracy: 0.0109 - loss: 5.8444
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 39s 366ms/step - accuracy: 0.0081 - loss: 5.1892
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 50s 461ms/step - accuracy: 0.0204 - loss: 4.8634
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 75s 382ms/step - accuracy: 0.0415 - loss: 4.6188
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 39s 365ms/step - accuracy: 0.0546 - loss: 4.4038
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 363ms/step - accuracy: 0.0697 - loss: 4.1912
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 33s 366ms/step - accuracy: 0.0942 - loss: 3.9337
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 40s 358ms/step - accuracy: 0.1361 - loss: 3.7461
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 361ms/step - accuracy: 0.1395 - loss: 3.6752
Epoch 10/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 363ms/step - accuracy: 0.1523 - loss: 3.7187
Epoch 11/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 42s 374ms/step - accuracy: 0.2047 - loss: 3.3535
Epoch 12/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 39

In [46]:
loss, accuracy = model.evaluate(X_test, encoded_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy  )

Test loss: 2.201958179473877
Test accuracy: 0.38119834661483765
